<a href="https://colab.research.google.com/github/TANZID36/Credit-Card-Fraud-Detection/blob/main/Credit_Card_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the Dependencies

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Data Collection and Preprocessing

In [2]:
#loading the dataset to a pandas DataFrame
credit_card_dataset = pd.read_csv('/content/drive/MyDrive/csv files for colab/creditcard.csv')

#dataset contains only numerical input variables which are the result of a PCA transformation (except time and money), since credit card informations are sensitive, the columns names are replaced with V1-28

In [3]:
credit_card_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
#number of rows and columns
credit_card_dataset.shape

(284807, 31)

In [5]:
#dataset information
credit_card_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [6]:
#checking the number of missing values
credit_card_dataset.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [7]:
#distribution of legit and fraud transactions
credit_card_dataset['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

The dataset is extremely unbalanced as the difference between legit and fraud transcation is huge

0 = legit 1 = fraud

In [8]:
#seperating the data for analysis
legit = credit_card_dataset[credit_card_dataset.Class == 0]
fraud = credit_card_dataset[credit_card_dataset.Class == 1]

In [9]:
print(legit.shape,fraud.shape)

(284315, 31) (492, 31)


In [10]:
#statistical measures of the data
legit.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [11]:
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [12]:
#comparing the values of both transaction
credit_card_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


Under Sampling

Building a sample dataset containing similar distribution of normal transactions and fraud transactions

randomly taking 492 transaction from legit and all 492 transaction from fraud, the purpose is to make a balanced 50-50 dataset, so the model can learn better

In [13]:
legit_sample = legit.sample(n=492)

Concatenating the two data frame

In [14]:
new_dataset = pd.concat([legit_sample, fraud], axis=0)

In [15]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
205006,135550.0,-2.013578,-0.271445,1.761347,-0.843163,1.437591,1.413172,0.577019,0.019395,1.111453,...,-0.533423,-0.037258,0.414666,-1.712116,0.370621,-0.306331,0.257596,-0.626588,21.58,0
49297,43994.0,0.809800,-2.037519,0.335043,-1.123377,-1.771536,-0.300325,-0.641867,-0.180483,-1.932474,...,-0.126034,-0.693216,-0.099160,0.072735,-0.031448,-0.419882,0.002368,0.087202,349.70,0
143242,85213.0,-0.674463,0.643184,1.970408,0.663635,-1.450084,-0.111669,-0.162489,0.175565,0.482744,...,-0.078190,0.107510,0.030430,0.817560,-0.929723,1.029679,-0.355631,-0.175511,57.12,0
1765,1362.0,-1.230366,-0.161113,-0.047379,-2.154059,2.965511,2.743305,-0.203573,0.903028,-0.206710,...,-0.148673,-0.667846,-0.050043,1.043124,0.587575,0.909127,-0.062772,0.121454,14.90,0
206979,136468.0,-1.091986,-0.002620,2.277141,1.120163,-1.154366,1.908973,0.333082,-1.059885,0.971078,...,0.551759,0.182763,-0.656274,-0.407241,0.064737,0.152552,-0.125605,-0.385123,265.50,0


In [16]:
new_dataset['Class'].value_counts()

0    492
1    492
Name: Class, dtype: int64

Now are dataset is uniformaly distributed

In [17]:
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,99015.766260,-0.161730,-0.003779,-0.144920,-0.006563,-0.106835,-0.006562,0.166609,0.036822,0.007540,...,-0.053579,-0.052666,0.028951,0.010265,-0.012194,-0.001620,0.003447,0.028611,0.033577,102.573232
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


Splitting dataset into Feature and Target

In [18]:
X = new_dataset.drop(columns=['Class','Time'], axis=1)
Y = new_dataset['Class']

In [19]:
print(X)

              V1        V2        V3        V4        V5        V6        V7  \
205006 -2.013578 -0.271445  1.761347 -0.843163  1.437591  1.413172  0.577019   
49297   0.809800 -2.037519  0.335043 -1.123377 -1.771536 -0.300325 -0.641867   
143242 -0.674463  0.643184  1.970408  0.663635 -1.450084 -0.111669 -0.162489   
1765   -1.230366 -0.161113 -0.047379 -2.154059  2.965511  2.743305 -0.203573   
206979 -1.091986 -0.002620  2.277141  1.120163 -1.154366  1.908973  0.333082   
...          ...       ...       ...       ...       ...       ...       ...   
279863 -1.927883  1.125653 -4.518331  1.749293 -1.566487 -2.010494 -0.882850   
280143  1.378559  1.289381 -5.004247  1.411850  0.442581 -1.326536 -1.413170   
280149 -0.676143  1.126366 -2.213700  0.468308 -1.120541 -0.003346 -2.234739   
281144 -3.113832  0.585864 -5.399730  1.817092 -0.840618 -2.943548 -2.208002   
281674  1.991976  0.158476 -2.583441  0.408670  1.151147 -0.096695  0.223050   

              V8        V9       V10  .

Splitting the data into training and testing data

In [20]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [21]:
print(X.shape, X_train.shape, X_test.shape)

(984, 29) (787, 29) (197, 29)


Model Training: Logistic Regression

In [22]:
model = LogisticRegression()

In [23]:
model.fit(X_train,Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

Model Evaluation: Accuracy Score

In [24]:
#accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print('Accuracy Score on Training Data : ', training_data_accuracy)

Accuracy Score on Training Data :  0.9529860228716646


In [25]:
#accuracy on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy Score on Test Data : ', test_data_accuracy)

Accuracy Score on Test Data :  0.9289340101522843


Building a Predictive System

In [26]:
from sklearn.utils.extmath import np_version
#taking a random input
input_data = (-0.436905071360625,0.918966212909322,0.92459077438817,-0.727219053596792,0.915678718106307,-0.127867352079254,0.707641607333935,0.0879623554672504,-0.66527135413364,-0.737979823596458,0.32409781346169,0.277192107214981,0.252624256310781,-0.291896460370468,-0.184520169327133,1.14317370716197,-0.92870926272403,0.680469592634687,0.0254364616880793,-0.0470212823165035,-0.194795823794671,-0.672637997017793,-0.156857514491897,-0.888386320943716,-0.342413218776576,-0.049026728633951,0.0796923991551505,0.131023789452311,0.89)

#changing the input data to numpy array
input_data_as_numpy_array = np.asarray(input_data)

#reshaping the data as we are predicting the label for only one instance
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

prediction = model.predict(input_data_reshaped)
#print(prediction)

if (prediction[0]==1):
  print('The Transaction is Fradulent')
else:
  print('The transaction is Legit')

The transaction is Legit


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"
